In [ ]:
import pandas as pd

# Cargar datos
df = pd.read_csv("/content/cine-chile-limpia-oficial.csv", sep=";")


# Echar un vistazo
df.head(5)

,nombre-pelicula,año,tipo-largometraje,duracion-min,tipo-filme,color,director-01,idioma,consol-idioma,lugar-prod,...,Columna231,Columna232,Columna233,Columna234,Columna235,Columna236,Columna237,Columna238,Columna239,Columna240
0,La industria del salitre,1915,Largometraje documental,NI,35 mm,"Blanco & negro, Muda",Francisco Caamaño,Español,Español,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,La industria del salitre,1915,Largometraje documental,NI,35 mm,"Blanco & negro, Muda",Francisco Caamaño,Español,Español,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,La baraja de la muerte (o El enigma de la call...,1916,Largometraje de ficción,100,35 mm,"Blanco & negro, Muda",Salvador Giambastiani,Español,Español,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,El hombre de acero,1917,Largometraje de ficción,NI,35 mm,"Blanco & negro, Muda",Jorge Délano «Coke»,Español,Español,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,La agonía de Arauco (o El olvido de los muertos),1917,Largometraje de ficción,NI,35 mm,"Blanco & negro, Muda",Gabriela Bussenius,Español,Español,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import altair as alt
import pandas as pd

# Asegurarse de que la columna 'pais-estreno' esté limpia (ya se hizo en la celda anterior, pero por seguridad)
df['pais-estreno'] = df['pais-estreno'].astype(str).str.strip()

# Definir 'estreno_internacional' (ya se hizo en la celda anterior)
df['estreno_internacional'] = (~df['pais-estreno'].isin(['Chile', 'NI']))

# Definir 'estreno_en_chile' como True si el país de estreno es 'Chile'
df['estreno_en_chile'] = (df['pais-estreno'] == 'Chile')

# Asegurarse de que la columna 'año' sea numérica (ya se hizo en la celda anterior)
df['año'] = pd.to_numeric(df['año'], errors='coerce')

# Filtrar películas con estreno internacional y agrupar por año para contar
international_releases_per_year = df[df['estreno_internacional'] == True]
international_releases_per_year = international_releases_per_year.groupby('año').size().reset_index(name='cantidad_peliculas_internacionales')

# Filtrar películas con estreno en Chile y agrupar por año para contar
chile_releases_per_year = df[df['estreno_en_chile'] == True]
chile_releases_per_year = chile_releases_per_year.groupby('año').size().reset_index(name='cantidad_peliculas_chile')

# Unir ambos DataFrames por 'año'
merged_df = pd.merge(international_releases_per_year, chile_releases_per_year, on='año', how='outer').fillna(0)

# Convertir las columnas de cantidad a enteros, ya que fillna(0) podría haberlas convertido a flotantes
merged_df['cantidad_peliculas_internacionales'] = merged_df['cantidad_peliculas_internacionales'].astype(int)
merged_df['cantidad_peliculas_chile'] = merged_df['cantidad_peliculas_chile'].astype(int)

# Eliminar años nulos si los hay después de la conversión a numérico
merged_df = merged_df.dropna(subset=['año'])
merged_df['año'] = merged_df['año'].astype(int)

# Capa 1: Línea para películas estrenadas en Chile (eje izquierdo)
line_chile = (
    alt.Chart(merged_df)
    .mark_line(point=True, color="#A52019")
    .encode(
        x=alt.X(
            "año:O",
            axis=alt.Axis(format="d"),
            title="Año de Estreno"
        ),
        y=alt.Y(
            "cantidad_peliculas_chile:Q",
            title="Películas estrenadas en Chile",
            axis=alt.Axis(titleColor="#A52019")
        ),
        tooltip=["año", "cantidad_peliculas_chile"]
    )
)

# Capa 2: Línea para películas estrenadas internacionalmente (eje derecho) con escala independiente
line_internacional = (
    alt.Chart(merged_df)
    .mark_line(point=True, color="#88937B")
    .encode(
        x="año:O",
        y=alt.Y(
            "cantidad_peliculas_internacionales:Q",
            title="Películas estrenadas internacionalmente",
            axis=alt.Axis(titleColor="#88937B"),
            scale=alt.Scale(
                domain=[0, merged_df['cantidad_peliculas_internacionales'].max() * 1.1] # Ajustar el dominio dinámicamente
            )
        ),
        tooltip=["año", "cantidad_peliculas_internacionales"]
    )
)

# Unir capas y resolver escalas independientemente
dual_axis_chart = (
    alt.layer(line_chile, line_internacional)
    .resolve_scale(y="independent")
    .properties(
        title='Evolución de películas chilenas por lugar de estreno'
    )
)

dual_axis_chart

alt.LayerChart(...)

In [ ]:
dual_axis_chart.save('evolucion_peliculas_chilenas.html')

El gráfico se ha guardado como `evolucion_peliculas_chilenas.html`.

In [ ]:
import altair as alt

# Limpiar la columna 'pais-estreno' eliminando espacios en blanco
df['pais-estreno'] = df['pais-estreno'].astype(str).str.strip()

# Definir 'estreno_internacional' como True si el país de estreno no es 'Chile' y no es 'NI'
df['estreno_internacional'] = (~df['pais-estreno'].isin(['Chile', 'NI']))

# Asegurarse de que la columna 'año' sea numérica
df['año'] = pd.to_numeric(df['año'], errors='coerce')

# Filtrar películas con estreno internacional y agrupar por año para contar
df_international_releases = df[df['estreno_internacional'] == True]
international_releases_per_year = df_international_releases.groupby('año').size().reset_index(name='cantidad_peliculas')

# Crear el gráfico de líneas con Altair
chart = alt.Chart(international_releases_per_year).mark_line(point=True).encode(
    x=alt.X('año:O', axis=alt.Axis(format="d"), title='Año de Estreno'), # 'O' para ordinal, para mostrar todos los años como categoría
    y=alt.Y('cantidad_peliculas:Q', title='Cantidad de Películas con Estreno Internacional'),
    tooltip=['año', 'cantidad_peliculas']
).properties(
    title='Evolución de Películas con Estreno Internacional por Año'
)

chart

alt.Chart(...)

In [ ]:
print(df['pais-estreno'].unique())

['Chile' 'NI' ' Francia' ' Chile' ' Argentina' ' Colombia' ' España'
 ' Estados Unidos' ' México' ' Cuba' ' Brasil' ' Italia' ' Villa Alemana'
 'Estados Unidos' ' Escocia' ' India' ' Canadá' ' Inglaterra' ' Perú'
 ' Alemania' ' Túnez' ' Holanda' ' Corea del Sur' ' República Checa']


In [ ]:
print(df.columns.tolist())

['nombre-pelicula', 'año', 'tipo-largometraje', 'duracion-min', 'tipo-filme', 'color', 'director-01', 'idioma', 'consol-idioma', 'lugar-prod', 'consol-lugar-prod', 'pais-rodaje', 'consol-pais-rodaje', 'lugar-estreno', 'pais-estreno', 'sinopsis', 'Columna1', 'Columna2', 'Columna3', 'Columna4', 'Columna5', 'Columna6', 'Columna7', 'Columna8', 'Columna9', 'Columna10', 'Columna11', 'Columna12', 'Columna13', 'Columna14', 'Columna15', 'Columna16', 'Columna17', 'Columna18', 'Columna19', 'Columna20', 'Columna21', 'Columna22', 'Columna23', 'Columna24', 'Columna25', 'Columna26', 'Columna27', 'Columna28', 'Columna29', 'Columna30', 'Columna31', 'Columna32', 'Columna33', 'Columna34', 'Columna35', 'Columna36', 'Columna37', 'Columna38', 'Columna39', 'Columna40', 'Columna41', 'Columna42', 'Columna43', 'Columna44', 'Columna45', 'Columna46', 'Columna47', 'Columna48', 'Columna49', 'Columna50', 'Columna51', 'Columna52', 'Columna53', 'Columna54', 'Columna55', 'Columna56', 'Columna57', 'Columna58', 'Columna5

In [ ]:
import altair as alt
import pandas as pd
import warnings

# Suprime advertencias de pandas comunes en Colab (como SettingWithCopyWarning)
warnings.filterwarnings('ignore')

# ----------------------------------------------------------------------
# FUNCIÓN PRINCIPAL PARA GENERAR GRÁFICOS DUALES POR GÉNERO
# ----------------------------------------------------------------------

def generar_grafico_dual_por_genero(df, genero_a_filtrar, titulo_grafico):
    """
    Filtra el DataFrame por género y genera un gráfico de línea con doble eje Y
    para comparar estrenos en Chile vs. estrenos internacionales.
    """

    # 1. FILTRAR DATOS POR GÉNERO
    # **¡IMPORTANTE! Reemplaza 'genero' por el nombre REAL de tu columna de género**
    df_genero = df[df['genero'] == genero_a_filtrar].copy()

    if df_genero.empty:
        print(f"Advertencia: No se encontraron datos para el género: {genero_a_filtrar}. No se generará el gráfico.")
        return None

    # 2. PREPARACIÓN DE LAS VARIABLES (Lógica exacta de tu código anterior)

    # Definir 'estreno_internacional' (True si NO es Chile o NI)
    df_genero['estreno_internacional'] = (~df_genero['pais-estreno'].astype(str).str.strip().isin(['Chile', 'NI']))

    # Definir 'estreno_en_chile' (True si el país de estreno es 'Chile')
    df_genero['estreno_en_chile'] = (df_genero['pais-estreno'].astype(str).str.strip() == 'Chile')

    # Asegurarse de que la columna 'año' sea numérica
    df_genero['año'] = pd.to_numeric(df_genero['año'], errors='coerce')

    # 3. CONTEO Y AGRUPACIÓN POR AÑO

    international_releases = (
        df_genero[df_genero['estreno_internacional'] == True]
        .groupby('año').size().reset_index(name='cantidad_peliculas_internacionales')
    )

    chile_releases = (
        df_genero[df_genero['estreno_en_chile'] == True]
        .groupby('año').size().reset_index(name='cantidad_peliculas_chile')
    )

    # 4. UNIR DATOS
    merged_df_genero = pd.merge(
        international_releases,
        chile_releases,
        on='año',
        how='outer'
    ).fillna(0)

    # Limpieza final
    merged_df_genero['cantidad_peliculas_internacionales'] = merged_df_genero['cantidad_peliculas_internacionales'].astype(int)
    merged_df_genero['cantidad_peliculas_chile'] = merged_df_genero['cantidad_peliculas_chile'].astype(int)
    merged_df_genero = merged_df_genero.dropna(subset=['año'])
    merged_df_genero['año'] = merged_df_genero['año'].astype(int)

    # 5. GENERACIÓN DEL GRÁFICO ALTAIR (LÓGICA DUAL-AXIS)

    # Calcular máximos para la escala dinámica de los ejes Y
    max_internacional = merged_df_genero['cantidad_peliculas_internacionales'].max()
    max_chile = merged_df_genero['cantidad_peliculas_chile'].max()

    # Colores (usando los de tu código anterior)
    COLOR_CHILE = "#A52019" # Rojo
    COLOR_INTERNACIONAL = "#88937B" # Verde/Grisáceo

    # Capa 1: Línea para estrenos en Chile (Eje Izquierdo)
    line_chile = (
        alt.Chart(merged_df_genero)
        .mark_line(point=True, color=COLOR_CHILE)
        .encode(
            x=alt.X(
                "año:O",
                axis=alt.Axis(format="d", labelAngle=-45, tickCount='exact'), # tickCount='exact' ayuda a mostrar todos
                title="Año de Estreno"
            ),
            y=alt.Y(
                "cantidad_peliculas_chile:Q",
                title="Estrenos en Chile (Películas)",
                axis=alt.Axis(titleColor=COLOR_CHILE, labelColor=COLOR_CHILE),
                scale=alt.Scale(domain=[0, max_chile * 1.1])
            ),
            tooltip=[alt.Tooltip("año:O", title="Año"),
                     alt.Tooltip("cantidad_peliculas_chile:Q", title="Estrenos en Chile")]
        )
    )

    # Capa 2: Línea para estrenos internacionales (Eje Derecho)
    line_internacional = (
        alt.Chart(merged_df_genero)
        .mark_line(point=True, color=COLOR_INTERNACIONAL)
        .encode(
            x="año:O", # Se comparte el eje X
            y=alt.Y(
                "cantidad_peliculas_internacionales:Q",
                title="Estrenos Internacionales (Películas)",
                axis=alt.Axis(titleColor=COLOR_INTERNACIONAL, labelColor=COLOR_INTERNACIONAL),
                scale=alt.Scale(domain=[0, max_internacional * 1.1])
            ),
            tooltip=[alt.Tooltip("año:O", title="Año"),
                     alt.Tooltip("cantidad_peliculas_internacionales:Q", title="Estrenos Internacionales")]
        )
    )

    # Unir capas, resolver escalas independientemente y configurar el título
    dual_axis_chart = (
        alt.layer(line_chile, line_internacional)
        .resolve_scale(y="independent")
        .properties(title=titulo_grafico)
        .interactive() # Permite zoom y desplazamiento
    )

    return dual_axis_chart

# ----------------------------------------------------------------------
# 3. EJECUCIÓN: GENERAR AMBOS GRÁFICOS
# ----------------------------------------------------------------------

# Llama a la función para el primer gráfico: FICCIÓN
# Asegúrate de que 'Ficción' sea el valor exacto en tu columna de género.
titulo_ficcion = 'Evolución de películas chilenas de FICCIÓN por lugar de estreno'
grafico_ficcion = generar_grafico_dual_por_genero(df, 'Ficción', titulo_ficcion)

# Llama a la función para el segundo gráfico: DOCUMENTAL
# Asegúrate de que 'Documental' sea el valor exacto en tu columna de género.
titulo_documental = 'Evolución de películas chilenas de DOCUMENTALES por lugar de estreno'
grafico_documental = generar_grafico_dual_por_genero(df, 'Documental', titulo_documental)


# 4. MOSTRAR AMBOS GRÁFICOS
print("--- GRÁFICO 1: FICCIÓN ---")
if grafico_ficcion:
    grafico_ficcion.display()

print("\n--- GRÁFICO 2: DOCUMENTALES ---")
if grafico_documental:
    grafico_documental.display()

In [7]:
import altair as alt
import pandas as pd
import warnings

# Suprime advertencias de pandas comunes en Colab (como SettingWithCopyWarning)
warnings.filterwarnings('ignore')

# Cargar datos para asegurar que df tenga las columnas esperadas
df = pd.read_csv("/content/cine-chile-limpia-oficial.csv", sep=";")

# ----------------------------------------------------------------------
# FUNCIÓN PRINCIPAL PARA GENERAR GRÁFICOS DUALES POR GÉNERO
# ----------------------------------------------------------------------

def generar_grafico_dual_por_genero(df, genero_a_filtrar, titulo_grafico):
    """
    Filtra el DataFrame por la columna 'tipo-largometraje' y genera un gráfico de
    línea con doble eje Y para comparar estrenos en Chile vs. internacionales.
    """

    # 1. FILTRAR DATOS POR GÉNERO
    # Usamos la columna correcta: 'tipo-largometraje'
    # Limpiamos los valores de esta columna para asegurar una coincidencia exacta
    df['tipo-largometraje'] = df['tipo-largometraje'].astype(str).str.strip()

    df_genero = df[df['tipo-largometraje'] == genero_a_filtrar].copy()

    if df_genero.empty:
        print(f"Advertencia: No se encontraron datos para el tipo de largometraje: {genero_a_filtrar}. No se generará el gráfico.")
        return None

    # 2. PREPARACIÓN DE LAS VARIABLES (Lógica exacta de tu código anterior)

    # Limpiar 'pais-estreno' antes de definir las flags
    df_genero['pais-estreno'] = df_genero['pais-estreno'].astype(str).str.strip()

    # Definir 'estreno_internacional' (True si NO es Chile o NI)
    df_genero['estreno_internacional'] = (~df_genero['pais-estreno'].isin(['Chile', 'NI']))

    # Definir 'estreno_en_chile' (True si el país de estreno es 'Chile')
    df_genero['estreno_en_chile'] = (df_genero['pais-estreno'] == 'Chile')

    # Asegurarse de que la columna 'año' sea numérica
    df_genero['año'] = pd.to_numeric(df_genero['año'], errors='coerce')

    # 3. CONTEO Y AGRUPACIÓN POR AÑO

    international_releases = (
        df_genero[df_genero['estreno_internacional'] == True]
        .groupby('año').size().reset_index(name='cantidad_peliculas_internacionales')
    )

    chile_releases = (
        df_genero[df_genero['estreno_en_chile'] == True]
        .groupby('año').size().reset_index(name='cantidad_peliculas_chile')
    )

    # 4. UNIR DATOS
    merged_df_genero = pd.merge(
        international_releases,
        chile_releases,
        on='año',
        how='outer'
    ).fillna(0)

    # Limpieza final
    merged_df_genero['cantidad_peliculas_internacionales'] = merged_df_genero['cantidad_peliculas_internacionales'].astype(int)
    merged_df_genero['cantidad_peliculas_chile'] = merged_df_genero['cantidad_peliculas_chile'].astype(int)
    merged_df_genero = merged_df_genero.dropna(subset=['año'])
    merged_df_genero['año'] = merged_df_genero['año'].astype(int)

    # 5. GENERACIÓN DEL GRÁFICO ALTAIR (LÓGICA DUAL-AXIS)

    # Calcular máximos para la escala dinámica de los ejes Y
    max_internacional = merged_df_genero['cantidad_peliculas_internacionales'].max()
    max_chile = merged_df_genero['cantidad_peliculas_chile'].max()

    # Colores (usando los de tu código anterior)
    COLOR_CHILE = "#A52019" # Rojo
    COLOR_INTERNACIONAL = "#88937B" # Verde/Grisáceo

    # Capa 1: Línea para estrenos en Chile (Eje Izquierdo)
    line_chile = (
        alt.Chart(merged_df_genero)
        .mark_line(point=True, color=COLOR_CHILE)
        .encode(
            x=alt.X(
                "año:O",
                axis=alt.Axis(format="d", labelAngle=-45),
                title="Año de Estreno"
            ),
            y=alt.Y(
                "cantidad_peliculas_chile:Q",
                title="Películas estrenadas en Chile",
                axis=alt.Axis(titleColor=COLOR_CHILE, labelColor=COLOR_CHILE),
                scale=alt.Scale(domain=[0, max_chile * 1.1])
            ),
            tooltip=[alt.Tooltip("año:O", title="Año"),
                     alt.Tooltip("cantidad_peliculas_chile:Q", title="Estrenos en Chile")]
        )
    )

    # Capa 2: Línea para estrenos internacionales (Eje Derecho)
    line_internacional = (
        alt.Chart(merged_df_genero)
        .mark_line(point=True, color=COLOR_INTERNACIONAL)
        .encode(
            x="año:O", # Se comparte el eje X
            y=alt.Y(
                "cantidad_peliculas_internacionales:Q",
                title="Películas estrenadas internacionalmente",
                axis=alt.Axis(titleColor=COLOR_INTERNACIONAL, labelColor=COLOR_INTERNACIONAL),
                scale=alt.Scale(domain=[0, max_internacional * 1.1])
            ),
            tooltip=[alt.Tooltip("año:O", title="Año"),
                     alt.Tooltip("cantidad_peliculas_internacionales:Q", title="Estrenos Internacionales")]
        )
    )

    # Unir capas, resolver escalas independientemente y configurar el título
    dual_axis_chart = (
        alt.layer(line_chile, line_internacional)
        .resolve_scale(y="independent")
        .properties(title=titulo_grafico)
        .interactive()
    )

    return dual_axis_chart

# ----------------------------------------------------------------------
# 3. EJECUCIÓN: GENERAR AMBOS GRÁFICOS
# ----------------------------------------------------------------------

# Get unique values from 'tipo-largometraje' to ensure correct filtering
print("Unique values in 'tipo-largometraje':", df['tipo-largometraje'].astype(str).str.strip().unique())

# Based on the unique values, we will use the correct genre names. Assuming they are 'Largometraje de ficción' and 'Largometraje documental'
# Gráfico 1: FICCIÓN
titulo_ficcion = 'Evolución de películas chilenas de Ficción por lugar de estreno'
grafico_ficcion = generar_grafico_dual_por_genero(df, 'Largometraje de ficción', titulo_ficcion)

# Gráfico 2: DOCUMENTALES
titulo_documental = 'Evolución de películas chilenas de Documentales por lugar de estreno'
grafico_documental = generar_grafico_dual_por_genero(df, 'Largometraje documental', titulo_documental)


# 4. MOSTRAR AMBOS GRÁFICOS
print("--- GRÁFICO 1: Ficción ---")
if grafico_ficcion:
    grafico_ficcion.display()

print("\n--- GRÁFICO 2: Documentales ---")
if grafico_documental:
    grafico_documental.display()

Unique values in 'tipo-largometraje': ['Largometraje documental' 'Largometraje de ficción']
--- GRÁFICO 1: Ficción ---


alt.LayerChart(...)


--- GRÁFICO 2: Documentales ---


alt.LayerChart(...)

In [8]:
# CÓDIGO PARA GUARDAR LOS GRÁFICOS COMO ARCHIVOS HTML

# ----------------------------------------------------------------------
# 1. Guardar Gráfico de FICCIÓN
# ----------------------------------------------------------------------

# Se utiliza el método .save() de Altair.
# Los archivos aparecerán en el explorador de archivos de Colab (carpeta a la izquierda).
if 'grafico_ficcion' in locals() and grafico_ficcion:
    try:
        nombre_archivo_ficcion = "evolucion_peliculas_FICCIÓN.html"
        grafico_ficcion.save(nombre_archivo_ficcion)
        print(f"✅ Gráfico de FICCIÓN guardado con éxito como: {nombre_archivo_ficcion}")
    except Exception as e:
        print(f"❌ Error al guardar el gráfico de Ficción: {e}")
else:
    print("❌ La variable 'grafico_ficcion' no está definida. Asegúrate de ejecutar la celda de generación de gráficos primero.")


# ----------------------------------------------------------------------
# 2. Guardar Gráfico de DOCUMENTALES
# ----------------------------------------------------------------------

if 'grafico_documental' in locals() and grafico_documental:
    try:
        nombre_archivo_documental = "evolucion_peliculas_DOCUMENTALES.html"
        grafico_documental.save(nombre_archivo_documental)
        print(f"✅ Gráfico de DOCUMENTALES guardado con éxito como: {nombre_archivo_documental}")
    except Exception as e:
        print(f"❌ Error al guardar el gráfico de Documentales: {e}")
else:
    print("❌ La variable 'grafico_documental' no está definida. Asegúrate de ejecutar la celda de generación de gráficos primero.")

✅ Gráfico de FICCIÓN guardado con éxito como: evolucion_peliculas_FICCIÓN.html
✅ Gráfico de DOCUMENTALES guardado con éxito como: evolucion_peliculas_DOCUMENTALES.html
